# **Day 8: Haunted Wasteland**

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

/Users/thubbard/Documents/Personal/Programming/advent-of-code-2023


Now, I'm going to import some libraries:

In [2]:
# Import statements
import pandas as pd
import re

Finally, I'll load in the data for this puzzle. 

In [3]:
# Load in the data for the puzzle
day = 8
input_data_path = f"data/input-files/day-{day:02d}-input.txt"
example_data_path = f"data/example-input/day-{day:02d}-example.txt"
with open(input_data_path, "r") as txt_file:
    input_data = txt_file.readlines()

# Parsing the Puzzle Input
This one seems deceptively easy... I'm a little suspicuous about Part 2. 

In [4]:
# Parse a DataFrame from the input_data
direction_sequence = input_data[0].strip()
node_info_str_list = [x.strip() for x in input_data[2:]]
node_info_df = pd.DataFrame.from_records(
    [
        {
            "node_name": node_info.split(" = ")[0],
            "left_node": node_info.split(" = ")[1].split(",")[0][1:],
            "right_node": node_info.split(" = ")[1].split(",")[1][1:-1],
        }
        for node_info in node_info_str_list
    ]
)

# Make a dictionary version of the data from the node_info_df
node_dict = {row.node_name: {
    "L": row.left_node,
    "R": row.right_node,
    } for row in node_info_df.itertuples()}

# Traversal Method
Now, I'm going to write a method that'll traverse the nodes from AAA to ZZZ. 

In [10]:
def traverse_nodes(directions, nodes):
    """
    This method will follow a set of directions through a set of nodes.
    """
    
    # Create a while loop 
    visited_nodes = 0
    visited_nodes_path = []
    cur_node = "AAA"
    while cur_node != "ZZZ": 
        cur_direction_idx = visited_nodes % len(directions)
        cur_direction = directions[cur_direction_idx]
        
        visited_nodes_path.append(cur_node)
        cur_node = nodes.get(cur_node).get(cur_direction)
        visited_nodes += 1
        print(f"On step {visited_nodes}, we're at index {cur_direction_idx}. The direction is {cur_direction}. cur_node = {cur_node}")

    # Return the number of nodes traveled, as well as the path 
    return visited_nodes, visited_nodes_path

# Run the input data through the method 
n_visited_nodes, visited_nodes_path = traverse_nodes(direction_sequence, node_dict)

# Print the number of visited nodes
print(f"You encounter '{n_visited_nodes}' nodes during your travels.")

On step 1, we're at index 0. The direction is L. cur_node = QNB
On step 2, we're at index 1. The direction is L. cur_node = GJN
On step 3, we're at index 2. The direction is L. cur_node = MLR
On step 4, we're at index 3. The direction is L. cur_node = GHT
On step 5, we're at index 4. The direction is R. cur_node = JMQ
On step 6, we're at index 5. The direction is L. cur_node = XPP
On step 7, we're at index 6. The direction is R. cur_node = PJL
On step 8, we're at index 7. The direction is L. cur_node = FPL
On step 9, we're at index 8. The direction is R. cur_node = HMK
On step 10, we're at index 9. The direction is R. cur_node = HSF
On step 11, we're at index 10. The direction is L. cur_node = SNP
On step 12, we're at index 11. The direction is R. cur_node = SHR
On step 13, we're at index 12. The direction is R. cur_node = VLH
On step 14, we're at index 13. The direction is R. cur_node = MSH
On step 15, we're at index 14. The direction is L. cur_node = MTV
On step 16, we're at index 15

In [6]:
len(direction_sequence)

307